In [1]:
import plotly.graph_objects as go
import os
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
import plotly.figure_factory as ff
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense

from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

2023-06-27 15:42:48.361522: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-27 15:42:48.362832: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-27 15:42:48.391600: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-27 15:42:48.392258: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 15:42:48.914171: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
import sys
sys.path.insert(1, '/home/krystian/Repos/XTB-Trader/utility')
sys.path.insert(1, '/home/krystian/Repos/XTB-Trader')
from XTBClient.api import XTBClient, PERIOD
import utility 

In [3]:
def get_data_for_symbol(symbol):
    c = XTBClient()
    c.login(os.environ.get("XTB_user_num"), os.environ.get("XTB_pass"))
    df = c.get_candles_in_range(symbol, PERIOD.FIFTEEN_MINUTES.value, start=datetime.now() - timedelta(days=365))
    c.logout()
    
    df = df.reset_index()
    df.rename(columns={"timestamp": "date"}, inplace=True)
    
    df['date'] = df['date'].apply(pd.to_datetime)
    df.set_index('date',inplace=True)
    return df

In [4]:
download_df = get_data_for_symbol("EURUSD")
download_df

,open,close,high,low,volume
date,,,,,
2022-12-26 23:00:00,1.06346,1.06352,1.06370,1.06286,2916.0
2022-12-26 23:15:00,1.06352,1.06380,1.06395,1.06344,3912.0
2022-12-26 23:30:00,1.06380,1.06444,1.06454,1.06380,4244.0
2022-12-26 23:45:00,1.06446,1.06395,1.06452,1.06389,2841.0
2022-12-27 00:00:00,1.06393,1.06423,1.06426,1.06341,7083.0
...,...,...,...,...,...
2023-06-27 12:30:00,1.09548,1.09704,1.09715,1.09443,18078.0
2023-06-27 12:45:00,1.09706,1.09647,1.09706,1.09585,15137.0
2023-06-27 13:00:00,1.09650,1.09644,1.09654,1.09612,14509.0


In [7]:
df = pd.DataFrame()
df['close'] = download_df['close']
df.index = pd.to_datetime(download_df.index)
df

,close
date,
2022-12-26 23:00:00,1.06352
2022-12-26 23:15:00,1.06380
2022-12-26 23:30:00,1.06444
2022-12-26 23:45:00,1.06395
2022-12-27 00:00:00,1.06423
...,...
2023-06-27 12:30:00,1.09704
2023-06-27 12:45:00,1.09647
2023-06-27 13:00:00,1.09644


In [21]:
def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
    first_date = pd.to_datetime(first_date_str)
    last_date = pd.to_datetime(last_date_str)

    target_date = first_date

    dates = []
    X, Y = [], []

    last_time = False
    while True:
        df_subset = dataframe.loc[:target_date].tail(n+1)

        values = df_subset['close'].to_numpy()
        x, y = values[:-1], values[-1]

        dates.append(target_date)
        np.append(X, x)
        np.append(Y, y)

        next_week = dataframe.loc[target_date:target_date+timedelta(days=7)]
        next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
        next_date_str = next_datetime_str.split('T')[0]
        year_month_day = next_date_str.split('-')
        year, month, day = year_month_day
        next_date = datetime(day=int(day), month=int(month), year=int(year))

        if last_time:
            break

        target_date = next_date

        if target_date == last_date:
            last_time = True

        ret_df = pd.DataFrame({})
        ret_df['Target Date'] = dates

        X = np.array(X)
        for i in range(0, n):
            ret_df[f'Target-{n-i}'] = X[:, i]

        ret_df['Target'] = Y

    return ret_df

# Start day second time around: '2021-03-25'
windowed_df = df_to_windowed_df(df, 
                                '2023-01-01', 
                                '2023-05-01', 
                                n=3)
windowed_df

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed